In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Set a professional style for plots
sns.set_style("whitegrid")
plt.rcParams['figure.dpi'] = 100 # Improve plot resolution

# Load the dataset
# Please adjust the file_path to where you have saved the CSV file
file_path = '/kaggle/input/daigt-v2-train-dataset/train_v2_drcat_02.csv' 
try:
    df = pd.read_csv(file_path)
    print("Dataset loaded successfully.")
except FileNotFoundError:
    print(f"Error: File not found at '{file_path}'. Please check the path.")
    # Create an empty DataFrame to prevent subsequent code from crashing
    df = pd.DataFrame()

if not df.empty:
    # 1. Display the first 5 rows to understand the structure
    print("\n--- 1. First 5 Rows of the Dataset ---")
    display(df.head())

    # 2. Get general information about the DataFrame (entry count, columns, non-null values, dtypes)
    print("\n\n--- 2. DataFrame General Information ---")
    df.info()

    # 3. Get descriptive statistics for numerical columns (in this case, 'label')
    print("\n\n--- 3. Descriptive Statistics ---")
    display(df.describe())

    # 4. Check for missing values in each column
    print("\n\n--- 4. Checking for Missing Values ---")
    missing_values = df.isnull().sum()
    if missing_values.sum() == 0:
        print("No missing values found in the dataset.")
    else:
        print(missing_values[missing_values > 0])


In [ ]:
if not df.empty:
    plt.figure(figsize=(10, 6))
    
    # Create a count plot for the label distribution
    ax = sns.countplot(x='label', data=df, palette=['#4374B3', '#FF6347'])
    
    plt.title('Distribution of Human-Written vs. AI-Generated Essays', fontsize=16)
    plt.xlabel('Label', fontsize=12)
    plt.ylabel('Count', fontsize=12)
    plt.xticks(ticks=[0, 1], labels=['Human (0)', 'AI-Generated (1)'])
    
    # Annotate the bars with the exact counts
    for p in ax.patches:
        ax.annotate(f'{p.get_height():,}', 
                    (p.get_x() + p.get_width() / 2., p.get_height()), 
                    ha='center', va='center', 
                    xytext=(0, 9), 
                    textcoords='offset points',
                    fontsize=11)
    
    plt.show()

    # Calculate the percentage of each class
    label_counts_perc = df['label'].value_counts(normalize=True) * 100
    print("\nPercentage Distribution of Labels:")
    print(f"Human-written essays (0): {label_counts_perc[0]:.2f}%")
    print(f"AI-generated essays (1): {label_counts_perc[1]:.2f}%")


- we do not have many problems using this dataset in case of data imbalance. We do not need any kind of oversampling tech in this project(that what I think now, maybe later we fix that)

In [ ]:
if not df.empty:
    plt.figure(figsize=(12, 10))
    
    # Plot the distribution of data sources
    source_order = df['source'].value_counts().index
    ax = sns.countplot(y='source', data=df, order=source_order, palette='viridis')
    
    plt.title('Distribution of Essays by Source', fontsize=16)
    plt.xlabel('Count (Log Scale)', fontsize=12)
    plt.ylabel('Source', fontsize=12)
    plt.xscale('log') # Use a log scale due to the large variance in counts
    
    # Add count labels to the bars
    for i, (p, count) in enumerate(zip(ax.patches, df['source'].value_counts(sort=True))):
        ax.text(p.get_width() * 1.1, i, f'{count:,}', va='center')
        
    plt.show()

    print("\nEssay Counts from Each Source:")
    print(df['source'].value_counts())


In [ ]:
if not df.empty:
    # Calculate word count for each essay
    df['word_count'] = df['text'].apply(lambda x: len(str(x).split()))

    plt.figure(figsize=(12, 7))
    
    # Plot the distribution of word counts for each class
    sns.histplot(data=df, x='word_count', hue='label', kde=True, bins=50, palette=['#4374B3', '#FF6347'])
    
    plt.title('Comparison of Essay Length Distribution (Word Count)', fontsize=16)
    plt.xlabel('Word Count', fontsize=12)
    plt.ylabel('Frequency', fontsize=12)
    
    # Manually create a legend
    handles, _ = plt.gca().get_legend_handles_labels()
    plt.legend(handles, ['AI-Generated (1)', 'Human (0)'], title='Label')

    plt.xlim(0, 1500) # Limit x-axis for better visualization
    plt.show()

    # Show descriptive statistics of word count grouped by label
    print("\nDescriptive Statistics for Word Count by Label:")
    display(df.groupby('label')['word_count'].describe())


In [ ]:
if not df.empty:
    # Due to the large number of unique prompts, let's visualize the top 15
    top_n_prompts = 15
    prompt_counts = df['prompt_name'].value_counts()
    
    plt.figure(figsize=(12, 8))
    sns.barplot(y=prompt_counts.index[:top_n_prompts], x=prompt_counts.values[:top_n_prompts], orient='h', palette='plasma')
    
    plt.title(f'Top {top_n_prompts} Most Frequent Essay Prompts', fontsize=16)
    plt.xlabel('Count', fontsize=12)
    plt.ylabel('Prompt Name', fontsize=12)
    plt.show()

    print(f"\nTotal number of unique prompts: {df['prompt_name'].nunique()}")
    print(f"\nTop {top_n_prompts} most frequent prompts:")
    print(prompt_counts.head(top_n_prompts))


- If a specific prompt_name (topic) is exclusively or heavily associated with only one label (AI or Human), the model might learn to use the topic as a shortcut for prediction.
- For example, if every essay about “Driverless cars” is AI-generated, your model might learn:

if text contains "driverless car", "autonomous vehicle", "self-driving" -> predict AI (1)

This model would fail spectacularly if it ever encountered a human-written essay on the same topic. Let’s systematically investigate this potential leakage.

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# We assume 'df' is already loaded from the previous steps
if not df.empty:
    print("--- Analyzing Label Distribution per Prompt Name ---")
    
    # fill_value=0 handles prompts that may only have one label type
    prompt_label_dist = df.groupby('prompt_name')['label'].value_counts().unstack(fill_value=0)
    
    # Rename columns for clarity
    prompt_label_dist = prompt_label_dist.rename(columns={0: 'human_count', 1: 'ai_count'})

    # Calculate the total count and the percentage of AI-generated essays for each prompt
    prompt_label_dist['total_count'] = prompt_label_dist['human_count'] + prompt_label_dist['ai_count']
    prompt_label_dist['ai_percentage'] = (prompt_label_dist['ai_count'] / prompt_label_dist['total_count']) * 100

    # Sort by the AI percentage to easily find the most skewed prompts
    prompt_label_dist_sorted = prompt_label_dist.sort_values(by='ai_percentage', ascending=False)

    print("\n--- Distribution of AI vs. Human Labels within each Prompt ---")
    print("Prompts are sorted by the percentage of AI-generated essays (from highest to lowest).")
    display(prompt_label_dist_sorted)

    # --- Identifying "Leaky" Prompts ---
    # Prompts that are 100% AI or 100% Human
    pure_ai_prompts = prompt_label_dist_sorted[prompt_label_dist_sorted['ai_percentage'] == 100]
    pure_human_prompts = prompt_label_dist_sorted[prompt_label_dist_sorted['ai_percentage'] == 0]

    # Prompts that are heavily skewed (e.g., > 95% AI or < 5% AI)
    highly_skewed_prompts = prompt_label_dist_sorted[
        (prompt_label_dist_sorted['ai_percentage'] > 95) | 
        (prompt_label_dist_sorted['ai_percentage'] < 5)
    ]

    print(f"\n\n--- Leakage Investigation Results ---")
    print(f"Total number of unique prompts: {len(prompt_label_dist)}")
    print(f"Number of prompts that are 100% AI-generated: {len(pure_ai_prompts)}")
    print(f"Number of prompts that are 100% Human-written: {len(pure_human_prompts)}")
    print(f"Total number of 'pure' (100% one label) or highly skewed (<5% or >95%) prompts: {len(highly_skewed_prompts)}")

    if len(pure_ai_prompts) > 0:
        print("\nPrompts that are 100% AI-Generated (High Risk of Leakage):")
        display(pure_ai_prompts.head()) # Display first few
    
    if len(pure_human_prompts) > 0:
        print("\nPrompts that are 100% Human-Written (High Risk of Leakage):")
        display(pure_human_prompts.head()) # Display first few

    # --- Visualization ---
    plt.figure(figsize=(12, 8))
    sns.histplot(prompt_label_dist['ai_percentage'], bins=20, kde=False)
    plt.title('Histogram of AI-Generated Percentage Across All Prompts', fontsize=16)
    plt.xlabel('Percentage of Essays that are AI-Generated (%)', fontsize=12)
    plt.ylabel('Number of Prompts', fontsize=12)
    plt.axvline(50, color='r', linestyle='--', label='50% (Balanced)')
    plt.axvline(95, color='orange', linestyle=':', label='95% Skew')
    plt.axvline(5, color='orange', linestyle=':', label='5% Skew')
    plt.legend()
    plt.show()
